In [1]:
import pandas as pd
import numpy as np

### Wczytanie danych

In [4]:
data = pd.read_csv('data/DataWorkshop dane do projektu.csv')

In [5]:
pd.set_option('display.max_columns', None)
data.sample(3)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,Skrzyżowanie,Geometria drogi,Charakterystyka miejsca zdarzenia,Oświetlenie,Warunki atmosferyczne,Rodzaj zdarzenia,Wina,Rodzaj drogi,Nawierzchnia drogi,Stan nawierzchni,Sygnalizacja świetlna,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków
46120,2016-11-20,23:00:00,18*40'322,50*15'321,GLIWICE,NaN,NaN,A4,"312,3",Obszar niezabudowany,NaN,Odcinek prosty,Jezdnia,Noc - droga oświetlona,Dobre warunki atmosferyczne,Najechanie na barierę ochronną,z winy kierującego,Autostrada,Twarda,Sucha,Brak,Jest,NaN,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,1,0
184043,2019-04-05,14:01:00,18*58'056,50*17'246,CHORZÓW,PARKOWA,20,P9423S,NaN,Obszar zabudowany,O ruchu okrężnym,NaN,Jezdnia,Światło dzienne,Dobre warunki atmosferyczne,Zderzenie pojazdów tylne,z winy kierującego,Jednokierunkowa,Twarda,Sucha,Brak,Jest,2.0,Niezachowanie bezp. odleg. między pojazdami,NaN,NaN,0,0,1,0
148369,2018-09-06,12:55:00,19*02'598,49*49'434,BIELSKO-BIAŁA,MOSTOWA,5,I2,NaN,Obszar zabudowany,NaN,Odcinek prosty,"Parking, plac, MOP",Światło dzienne,Dobre warunki atmosferyczne,Najechanie na pojazd unieruchomiony,z winy kierującego,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Nie ma,2.0,Nieprawidłowe omijanie,NaN,NaN,0,0,1,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229929 entries, 0 to 229928
Data columns (total 30 columns):
Data zdarzenia                       229929 non-null object
Godzina                              229929 non-null object
GPS x                                229929 non-null object
GPS y                                229929 non-null object
Miejscowość                          229929 non-null object
Ulica                                209201 non-null object
Numer domu                           136045 non-null object
Numer drogi                          229929 non-null object
KM HM (Pikietaż)                     35041 non-null object
Obszar                               229928 non-null object
Skrzyżowanie                         63150 non-null object
Geometria drogi                      163921 non-null object
Charakterystyka miejsca zdarzenia    229927 non-null object
Oświetlenie                          229929 non-null object
Warunki atmosferyczne                229928 non-nul

### Functions

In [7]:
#string conversion from Ani: 
def simplify(a):        
    a = a.replace('\'','')                   # usuń znaki '
    a = a.replace('\n','')                   # usuń znaki nowej linii
    a = a.replace(';','')                    # usuń średniki
    a = a.replace(',','')                    # usuń przecinki
    a = a.replace('.',' ')                   # zamień kropki na spacje (ze względu na dziwny zapis w adresach)
    a = a.replace('  ',' ')                  # usuń podwójne spacje
    a = a.lower()                            # zamień wielkie litery na małe
    a = a.replace('(od 11 2015)','')         # usuń ciąg "(od 11.2015)"
    a =' '.join(unique_list(a.split()))       # usuń zduplikowane słowa
    return(a)

def simplify_cell(string):
    return(simplify(str(string)))

def unique_list(l):
    ulist = []
    [ulist.append(x) for x in l if x not in ulist]
    return ulist   

def check_missing(df):
    for column in df.columns:
        missing = column, df[column].isnull().sum()
        if missing[1] == 0: continue
        print(missing)
        
def study_column(nazwa):
    print(df[nazwa].value_counts(normalize = True))
    print('wartości unikalne:')
    print(simplify_cell(np.array2string(df[nazwa].unique())))

#change words 
def make_one_word (i,a):
    b = a.replace(' ','_') 
    df[i] = df[i].str.replace(a,b, regex=True)
    
def word_or_word (x,word1,word2):
    if word1 in str(x):
        return word1 
    elif word2 in str(x):
        return word2 
    else: 
        return '-1' 

In [8]:
#Clone data
df = data.copy()

### Data gaps

In [9]:
check_missing(df)

('Ulica', 20728)
('Numer domu', 93884)
('KM HM (Pikietaż)', 194888)
('Obszar', 1)
('Skrzyżowanie', 166779)
('Geometria drogi', 66008)
('Charakterystyka miejsca zdarzenia', 2)
('Warunki atmosferyczne', 1)
('Rodzaj drogi', 1)
('Nawierzchnia drogi', 1)
('Stan nawierzchni', 1)
('Oznakowanie poziome', 2)
('Liczba pasów w zdarzeniu', 88154)
('Zachowanie kierującego', 27901)
('Zachowanie pieszego', 227444)
('Inne przyczyny', 203726)


In [10]:
df = df.fillna(-1) 

In [11]:
#GPS
# from Krzysztofa
import re
# https://stackoverflow.com/questions/33997361
def dms2dd(coord):
    """ GPS HH.MM.SS.S to DD (Decimal Degrees) conversion """
    c = re.findall(r"\w+", coord)
    degrees = float(c[0])
    minutes = float(c[1])
    if len(c) > 2:
        seconds = float(c[2])/10   
    else:
        seconds = 0
    dd = (float(degrees) + (minutes)/60 + float(seconds)/(60*60))
    return dd
df['gps_x'] = df['GPS x'].apply(lambda x : dms2dd(x))
df['gps_y'] = df['GPS y'].apply(lambda x : dms2dd(x))

del df['GPS x']
del df['GPS y']

In [17]:
df.head()

,Data zdarzenia,Godzina,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,Skrzyżowanie,Geometria drogi,Charakterystyka miejsca zdarzenia,Oświetlenie,Warunki atmosferyczne,Rodzaj zdarzenia,Wina,Rodzaj drogi,Nawierzchnia drogi,Stan nawierzchni,Sygnalizacja świetlna,Oznakowanie poziome,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,gps_x,gps_y,liczba_pasów_w_zdarzeniu,liczba_rannych_w_zdarzeniu,liczba_zabitych_w_zdarzeniu,liczba_kolizji,liczba_wypadków
0,2016-01-01,05:15:00,GLIWICE,TOSZECKA,-1,901,"61,6",Obszar niezabudowany,-1,Odcinek prosty,Jezdnia,Noc - droga nieoświetlona,Dobre warunki atmosferyczne,Najechanie na zwierzę,z innych przyczyn,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Jest,-1,-1,"Obiekty, zwierzęta na drodze",18.635806,50.369722,2.0,0.0,0.0,1.0,0.0
1,2016-01-01,00:05:00,DĄBROWA GÓRNICZA,SIENKIEWICZA,-1,G200 030S,-1,Obszar zabudowany,Z drogą z pierwsz.,-1,Jezdnia,Noc - droga oświetlona,Dobre warunki atmosferyczne,Najechanie na pojazd unieruchomiony,z winy kierującego,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Jest,Nieprawidłowe: cofanie,-1,-1,19.180833,50.323083,-1.0,0.0,0.0,1.0,0.0
2,2016-01-01,18:30:00,TARNOWSKIE GÓRY,KS.IGNACEGO SIWCA,4A,GMINNA,-1,Obszar zabudowany,-1,"Zakręt, łuk",Jezdnia,Noc - droga oświetlona,Dobre warunki atmosferyczne,Najechanie na pojazd unieruchomiony,z winy kierującego,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Nie ma,Nieprawidłowe: omijanie,-1,-1,18.877639,50.445083,2.0,0.0,0.0,1.0,0.0
3,2016-01-01,15:10:00,TARNOWSKIE GÓRY,JÓZEFA PIŁSUDSKIEGO,6,P3276S,-1,Obszar zabudowany,-1,Odcinek prosty,Jezdnia,Światło dzienne,Pochmurno,Zderzenie pojazdów tylne,z winy kierującego,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Jest,Nieprawidłowe: omijanie,-1,-1,18.860583,50.444667,2.0,0.0,0.0,1.0,0.0
4,2016-01-01,17:45:00,OŻAROWICE,DWORCOWA,16,P3203S,-1,Obszar zabudowany,-1,Odcinek prosty,Jezdnia,"Zmrok, świt",Dobre warunki atmosferyczne,Najechanie na zwierzę,z innych przyczyn,Jednojezdniowa dwukierunkowa,Twarda,Sucha,Brak,Nie ma,-1,-1,"Obiekty, zwierzęta na drodze",19.047556,50.466806,2.0,0.0,0.0,1.0,0.0


### Data type change

In [18]:
#float: 
for i in ['liczba_pasów_w_zdarzeniu','liczba_rannych_w_zdarzeniu','liczba_zabitych_w_zdarzeniu','liczba_kolizji',	'liczba_wypadków']:
    df[i.lower().replace(' ','_')] = df[i].astype(float)
    del df[i]

In [19]:
#string
for i in df.select_dtypes(include=[np.object]).columns:
    df[i.lower().replace(' ','_')] = df[i].apply(lambda x: simplify_cell(x))
    del df[i]

In [20]:
#time
df['data'] =  pd.to_datetime( df['data_zdarzenia'] + ' ' + df['godzina'], format='%Y-%m-%d %H:%M:%S.%f')
del df['data_zdarzenia']
del df['godzina']

In [21]:
df.sample(3)

,gps_x,gps_y,miejscowość,ulica,numer_domu,numer_drogi,km_hm_(pikietaż),obszar,skrzyżowanie,geometria_drogi,charakterystyka_miejsca_zdarzenia,oświetlenie,warunki_atmosferyczne,rodzaj_zdarzenia,wina,rodzaj_drogi,nawierzchnia_drogi,stan_nawierzchni,sygnalizacja_świetlna,oznakowanie_poziome,zachowanie_kierującego,zachowanie_pieszego,inne_przyczyny,data
10963,19.202222,50.293250,sosnowiec,szymanowskiego,-1,p7624s,-1,obszar zabudowany,z drogą pierwsz,-1,jezdnia,światło dzienne,dobre warunki atmosferyczne,zderzenie pojazdów boczne,z winy kierującego,dwie jezdnie jednokierunkowe,twarda,sucha,brak,nie ma,nieudzielenie pierwszeństwa przejazdu,-1,-1,2016-03-25 16:45:00
102225,19.248278,50.317639,dąbrowa górnicza,katowicka,-1,k94b,286,obszar niezabudowany,-1,odcinek prosty,jezdnia,światło dzienne,pochmurno opady deszczu,zderzenie pojazdów tylne,z winy kierującego,dwie jezdnie jednokierunkowe,twarda,mokra,brak,jest,niedostosowanie prędkości do warunków ruchu,-1,-1,2017-11-13 13:35:00
175309,18.551028,50.255917,sośnicowice,smolnicka,-1,p2916s,-1,obszar zabudowany,z drogą pierwsz,-1,jezdnia,światło dzienne,dobre warunki atmosferyczne,zderzenie pojazdów boczne,z winy kierującego,jednojezdniowa dwukierunkowa,twarda,mokra,brak,nie ma,nieustąpienie pierwszeństwa przejazdu,-1,-1,2019-02-06 11:00:00


In [21]:
#KM HM (pikietaż)
df['km_hm_pikietaż'] = df['km_hm_(pikietaż)'].str.replace(',','.', regex=True).astype(float)
del df['km_hm_(pikietaż)']

### Analysis of individual columns

In [22]:
for i in df.select_dtypes(include=[np.object]).columns:
    print(i)
    print(df[i].unique())
    print('----------------')

miejscowość
['gliwice' 'dąbrowa górnicza' 'tarnowskie góry' ... 'nowa rybna' 'borowce'
 'zberezka']
----------------
ulica
['toszecka' 'sienkiewicza' 'ks ignacego siwca' ...
 'władysława żeleńskiego' 'jana demarczyka' 'mędralowa']
----------------
numer_domu
['-1' '4a' '6' ... '4286' '167e' '1n']
----------------
numer_drogi
['901' 'g200 030s' 'gminna' ... 'g520313s' 'g380018s' 'g611078s']
----------------
km_hm_(pikietaż)
['616' '-1' '224' ... '3555' '1356' '2459']
----------------
obszar
['obszar niezabudowany' 'obszar zabudowany' '-1']
----------------
skrzyżowanie
['-1' 'z drogą pierwsz' 'o ruchu okrężnym' 'równorzędne']
----------------
geometria_drogi
['odcinek prosty' '-1' 'zakręt łuk' 'spadek' 'wzniesienie'
 'wierzchołek wzn' 'zakręt łuk spadek' 'odcinek prosty spadek'
 'zakręt łuk wzniesienie' 'odcinek prosty wzniesienie'
 'zakręt łuk wierzchołek wzn' 'odcinek prosty wierzchołek wzn'
 'odcinek prosty zakręt łuk']
----------------
charakterystyka_miejsca_zdarzenia
['jezdnia' 'c

In [23]:
df[df['skrzyżowanie']!= '-1']['geometria_drogi'].unique()
# you can combine the junction columns with the road geometry

array(['-1'], dtype=object)

In [24]:
df['geometria_drogi'] = df['geometria_drogi'].apply(lambda x: word_or_word (x,'odcinek prosty','zakręt łuk'))

In [25]:
df['typ_drogi'] = df.apply(lambda row: row['skrzyżowanie'] if row['geometria_drogi'] == '-1' else row["geometria_drogi"], axis=1)
del df['skrzyżowanie']
del df['geometria_drogi']

In [26]:
study_column('typ_drogi')

odcinek prosty      0.624014
z drogą pierwsz     0.240270
zakręt łuk          0.074845
o ruchu okrężnym    0.027274
-1                  0.026491
równorzędne         0.007107
Name: typ_drogi, dtype: float64
wartości unikalne:
[odcinek prosty z drogą pierwsz zakręt łuk o ruchu okrężnym równorzędne -1]


In [27]:
pd.DatetimeIndex(df[df['oświetlenie'] == 'świt zmrok']['data']).hour.unique()
pd.DatetimeIndex(df[df['oświetlenie'] == 'zmrok świt']['data']).hour.unique()
# it doesn't matter

df['oświetlenie'] = df['oświetlenie'].apply(lambda x: 'świt zmrok' if (x == 'świt zmrok' or x == 'zmrok świt' ) else x )

In [28]:
study_column('warunki_atmosferyczne')

dobre warunki atmosferyczne                         0.631347
pochmurno                                           0.189258
opady deszczu                                       0.103280
opady śniegu gradu                                  0.027565
pochmurno opady deszczu                             0.022198
oślepiające słońce                                  0.012386
mgła dym                                            0.004493
pochmurno opady śniegu gradu                        0.003223
silny wiatr                                         0.002322
silny wiatr opady deszczu                           0.000861
silny wiatr pochmurno                               0.000687
opady deszczu śniegu gradu                          0.000687
pochmurno mgła dym                                  0.000526
pochmurno opady deszczu śniegu gradu                0.000317
silny wiatr pochmurno opady deszczu                 0.000317
silny wiatr opady śniegu gradu                      0.000183
opady deszczu mgła dym  

In [29]:
make_one_word('warunki_atmosferyczne','dobre warunki atmosferyczne')
make_one_word('warunki_atmosferyczne','mgła dym')
make_one_word('warunki_atmosferyczne','oślepiające słońce')
make_one_word('warunki_atmosferyczne','silny wiatr')

In [30]:
for i in ['dobre_warunki_atmosferyczne','pochmurno', 'mgła_dym', 'oślepiające_słońce','śniegu','gradu', 'deszczu', 'silny_wiatr']:
    df[i] = df['warunki_atmosferyczne'].apply(lambda x: 1 if i in str(x) else 0)

In [31]:
study_column('stan_nawierzchni')

sucha                                                                       0.675696
mokra                                                                       0.271840
oblodzenie zaśnieżenie                                                      0.034002
oblodzona zaśnieżona                                                        0.004501
kałuże rozlewiska                                                           0.002488
sucha dziury wyboje                                                         0.002162
mokra oblodzenie zaśnieżenie                                                0.001953
mokra kałuże rozlewiska                                                     0.001796
dziury wyboje                                                               0.001257
zanieczyszczona                                                             0.000987
mokra dziury wyboje                                                         0.000918
sucha koleiny garby                                              

In [32]:
df['stan_nawierzchni'] = df['stan_nawierzchni'].apply(lambda x: 'zaśnieżona' if ('zaśnieżona' in str(x) or 'zaśnieżenie' in str(x) ) else x)
df['stan_nawierzchni'] = df['stan_nawierzchni'].apply(lambda x: 'koleiny' if ('koleiny' in str(x) or 'garby' in str(x) or 'dziury' is str(x) or 'kałuże' in str(x) or 'wyboje' in str(x)) else x)
df['stan_nawierzchni'] = df['stan_nawierzchni'].apply(lambda x: 'zanieczyszczona' if 'zanieczyszczona' in str(x) else x)                                             
                                                                               

In [33]:
study_column('inne_przyczyny')

-1                                         0.886039
obiekty zwierzęta na drodze                0.050463
nieustalone                                0.025556
stan jezdni                                0.020998
inne                                       0.008742
niewłaściwy stan jezdni                    0.002227
niesprawność techniczna pojazdu            0.001483
zabezpieczenie robót na drodze             0.001374
utrata przytomności śmierć kierującego     0.001157
z winy pasażera                            0.000948
pożar pojazdu                              0.000509
oślepienie przez inny pojazd lub słońce    0.000309
organizacja ruchu                          0.000083
nieprawidłowo zabezp roboty drogowe        0.000078
nieprawidłowo działająca zapora kolej      0.000022
nieprawidłowo działająca zapora rogatka    0.000009
działanie sygnalizacji świetlnej           0.000004
Name: inne_przyczyny, dtype: float64
wartości unikalne:
[obiekty zwierzęta na drodze -1 nieustalone niewłaściwy stan

In [34]:
df['inne_przyczyny'] = df['inne_przyczyny'].apply(lambda x: '-1' if (x == 'nieustalone' or x == 'inne' ) else x )

In [35]:
df[df['inne_przyczyny'] != '-1'][['wina','zachowanie_kierującego','zachowanie_pieszego','inne_przyczyny']]

,wina,zachowanie_kierującego,zachowanie_pieszego,inne_przyczyny
0,z innych przyczyn,-1,-1,obiekty zwierzęta na drodze
4,z innych przyczyn,-1,-1,obiekty zwierzęta na drodze
20,z innych przyczyn,-1,-1,niewłaściwy stan jezdni
22,z innych przyczyn,-1,-1,obiekty zwierzęta na drodze
23,z innych przyczyn,-1,-1,obiekty zwierzęta na drodze
...,...,...,...,...
229889,z innych przyczyn,-1,-1,stan jezdni
229891,z innych przyczyn,-1,-1,obiekty zwierzęta na drodze
229899,z innych przyczyn,-1,-1,stan jezdni
229900,z innych przyczyn,-1,-1,stan jezdni


In [36]:
df[df['zachowanie_pieszego']=='inne przyczyny']['inne_przyczyny'].unique() 

array(['-1', 'obiekty zwierzęta na drodze', 'organizacja ruchu',
       'zabezpieczenie robót na drodze', 'stan jezdni'], dtype=object)

In [37]:
df[df['zachowanie_kierującego']=='inne przyczyny']['inne_przyczyny'].unique() 

array(['-1', 'niewłaściwy stan jezdni', 'niesprawność techniczna pojazdu',
       'obiekty zwierzęta na drodze', 'z winy pasażera', 'pożar pojazdu',
       'stan jezdni', 'zabezpieczenie robót na drodze',
       'utrata przytomności śmierć kierującego',
       'oślepienie przez inny pojazd lub słońce'], dtype=object)

In [38]:
df[df['wina']=='z innych przyczyn']['inne_przyczyny'].unique() #here it fits best

array(['obiekty zwierzęta na drodze', 'niewłaściwy stan jezdni', '-1',
       'utrata przytomności śmierć kierującego', 'pożar pojazdu',
       'nieprawidłowo zabezp roboty drogowe',
       'niesprawność techniczna pojazdu', 'z winy pasażera',
       'oślepienie przez inny pojazd lub słońce',
       'nieprawidłowo działająca zapora rogatka', 'stan jezdni',
       'zabezpieczenie robót na drodze', 'organizacja ruchu',
       'nieprawidłowo działająca zapora kolej',
       'działanie sygnalizacji świetlnej'], dtype=object)

In [39]:
df['wina'] = df.apply(lambda x: x['inne_przyczyny'] if (x['inne_przyczyny'] != '-1') else x['wina'], axis=1)
del df['inne_przyczyny']

In [40]:
study_column('wina')

z winy kierującego                         0.863149
obiekty zwierzęta na drodze                0.050463
z innych przyczyn                          0.032601
stan jezdni                                0.020998
współwina uczestników ruchu                0.014357
z winy pieszego                            0.010229
niewłaściwy stan jezdni                    0.002227
niesprawność techniczna pojazdu            0.001483
zabezpieczenie robót na drodze             0.001374
utrata przytomności śmierć kierującego     0.001157
z winy pasażera                            0.000948
pożar pojazdu                              0.000509
oślepienie przez inny pojazd lub słońce    0.000309
organizacja ruchu                          0.000083
nieprawidłowo zabezp roboty drogowe        0.000078
nieprawidłowo działająca zapora kolej      0.000022
nieprawidłowo działająca zapora rogatka    0.000009
działanie sygnalizacji świetlnej           0.000004
Name: wina, dtype: float64
wartości unikalne:
[obiekty zwierzęta

In [93]:
# TODO: 
study_column('zachowanie_kierującego')

niezachowanie bezp odleg między pojazdami                                                                   0.186279
nieustąpienie pierwszeństwa przejazdu                                                                       0.149672
-1                                                                                                          0.121346
niedostosowanie prędkości do warunków ruchu                                                                 0.100879
nieprawidłowe cofanie                                                                                       0.093338
                                                                                                              ...   
nieprawidłowe: skręcanie zatrzymywanie postój                                                               0.000004
nieudzielenie pierwszeństwa przejazdu wyprzedzanie pojazdu przed przejściem dla pieszych                    0.000004
nieustąpienie pierwszeństwa pieszemu na przejściu dla pieszych n

In [94]:
len(df['zachowanie_kierującego'].unique())

124

In [95]:
df['zachowanie_kierującego'] = df['zachowanie_kierującego'].apply(lambda x: x.replace('nieprawidłowe ','')  ) 
df['zachowanie_kierującego'] = df['zachowanie_kierującego'].apply(lambda x: x.replace('nieprawidłowe:','')  )

### Sprawdzenie GPS 

In [41]:
from keplergl import KeplerGl 
map_1 = KeplerGl(height=500)
map_1.add_data(data=df, name='data')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [42]:
map_1

KeplerGl(data={'data': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21…

### save_data

In [43]:
df.sample(3)

,gps_x,gps_y,liczba_pasów_w_zdarzeniu,liczba_rannych_w_zdarzeniu,liczba_zabitych_w_zdarzeniu,liczba_kolizji,liczba_wypadków,miejscowość,ulica,numer_domu,numer_drogi,obszar,charakterystyka_miejsca_zdarzenia,oświetlenie,warunki_atmosferyczne,rodzaj_zdarzenia,wina,rodzaj_drogi,nawierzchnia_drogi,stan_nawierzchni,sygnalizacja_świetlna,oznakowanie_poziome,zachowanie_kierującego,zachowanie_pieszego,data,km_hm_pikietaż,typ_drogi,dobre_warunki_atmosferyczne,pochmurno,mgła_dym,oślepiające_słońce,śniegu,gradu,deszczu,silny_wiatr
116226,18.931889,50.196222,5.0,0.0,0.0,1.0,0.0,katowice,tadeusza kościuszki,-1,81,obszar niezabudowany,jezdnia,światło dzienne,dobre_warunki_atmosferyczne,zderzenie pojazdów boczne,z winy kierującego,dwie jezdnie jednokierunkowe,twarda,sucha,jest działa,jest,nieustąpienie pierwszeństwa przejazdu,-1,2018-02-06 16:30:00,-1.0,z drogą pierwsz,1,0,0,0,0,0,0,0
185141,18.654889,50.125333,2.0,0.0,0.0,1.0,0.0,przegędza,mikołowska,-1,w925,obszar zabudowany,jezdnia,światło dzienne,opady deszczu,zderzenie pojazdów boczne,z winy kierującego,jednojezdniowa dwukierunkowa,twarda,mokra,brak,jest,nieustąpienie pierwszeństwa przejazdu,-1,2019-04-12 13:10:00,-1.0,z drogą pierwsz,0,0,0,0,0,0,1,0
191000,19.024028,50.253056,2.0,0.0,0.0,1.0,0.0,katowice,lompy,-1,g100406s,obszar zabudowany,jezdnia,światło dzienne,dobre_warunki_atmosferyczne,najechanie na pojazd unieruchomiony,z winy kierującego,jednojezdniowa dwukierunkowa,twarda,sucha,brak,jest,nieprawidłowe omijanie,-1,2019-05-17 11:50:00,-1.0,odcinek prosty,1,0,0,0,0,0,0,0


In [44]:
df.to_csv('df_modified.csv')